In [1]:
import numpy as np

In [2]:
from ase.build import bulk
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import LBFGS

from disalloy.cellfunc import jitter_ic, jitter_ia, compute_distortion

In [3]:
from ase.io.espresso import read_espresso_out

In [4]:

# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='/qe/q-e/bin/pw.x', pseudo_dir='/qe/q-e/pseudo/SSSP/'
)

# Pseudopotentials from SSSP Efficiency v1.3.0
pseudopotentials = {'Na': 'na_pbe_v1.5.uspp.F.UPF', 'Cl': 'cl_pbe_v1.4.uspp.F.UPF'}

input_data = {
    'system': {'ecutwfc': 60, 'ecutrho': 480},
    'disk_io': 'low',  # Automatically put into the 'control' section
    'tstress':True,  # deprecated, put in input_data
    'tprnfor':True,  # deprecated, put in input_data
    'calculation': 'vc-relax',#'scf',
    'conv_thr':1e-8,
    'occupations':'smearing',
    'smearing':'mp',
    'degauss':0.02,
    'mixing_beta':0.7
}

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    input_data=input_data,
    kspacing=0.5e-1,
)


In [5]:

rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=3.0)
print(rocksalt)
rocksalt = jitter_ic(rocksalt, 0.005, seed =42)
print(rocksalt)

original = rocksalt.copy()
# rocksalt = jitter_ia(rocksalt, 0.01, seed =42)
# print(rocksalt)


Atoms(symbols='NaCl', pbc=True, cell=[[0.0, 1.5, 1.5], [1.5, 0.0, 1.5], [1.5, 1.5, 0.0]])
Atoms(symbols='NaCl', pbc=True, cell=[[0.0024845394287546594, 1.5055814258575095, 1.5017482144067107], [1.5046623175410332, 0.009159886555968918, 1.5065152844599097], [1.5004673986035537, 1.4944905293990984, 0.00043162900763017557]])


In [6]:
original.cell.array

array([[2.48453943e-03, 1.50558143e+00, 1.50174821e+00],
       [1.50466232e+00, 9.15988656e-03, 1.50651528e+00],
       [1.50046740e+00, 1.49449053e+00, 4.31629008e-04]])

In [7]:
rocksalt.cell.array

array([[2.48453943e-03, 1.50558143e+00, 1.50174821e+00],
       [1.50466232e+00, 9.15988656e-03, 1.50651528e+00],
       [1.50046740e+00, 1.49449053e+00, 4.31629008e-04]])

In [8]:
bohr = 0.529177211 # A

In [9]:
rocksalt.calc = calc

rocksalt.get_potential_energy()  # This will run a single point calculation

-1752.1577976160108

In [10]:
original.cell.array

array([[2.48453943e-03, 1.50558143e+00, 1.50174821e+00],
       [1.50466232e+00, 9.15988656e-03, 1.50651528e+00],
       [1.50046740e+00, 1.49449053e+00, 4.31629008e-04]])

In [11]:
rocksalt.cell.array

array([[2.48453943e-03, 1.50558143e+00, 1.50174821e+00],
       [1.50466232e+00, 9.15988656e-03, 1.50651528e+00],
       [1.50046740e+00, 1.49449053e+00, 4.31629008e-04]])

In [12]:
compute_distortion(original, rocksalt)

2.4709307555527296e-16

In [13]:
original.cell.cellpar()

array([ 1.12503355,  1.12648202,  1.12040591, 59.74403861, 59.95975318,
       59.76578559])

In [14]:
rocksalt.cell.cellpar()

array([ 1.12503355,  1.12648202,  1.12040591, 59.74403861, 59.95975318,
       59.76578559])

In [26]:
# bohr = 0.529177211 # A
with open('espresso.pwo', 'r') as f:
    for i in read_espresso_out(f,):
        struc_ite = i.copy()
        print(struc_ite.cell.array)
        print(compute_distortion(original, struc_ite))
        # print(i.get_forces())
        # print(i.get_stress())
        print(i.get_potential_energy())
        print()


[[2.48375965e-03 1.50558152e+00 1.50174743e+00]
 [1.50466287e+00 9.15886371e-03 1.50651506e+00]
 [1.50046727e+00 1.49448966e+00 4.31680830e-04]]
9.89377308334673e-07
-1682.9705830757152

[[0.0039505  2.06448483 2.04913642]
 [2.05586592 0.01351675 2.05523901]
 [2.04881727 2.04756406 0.00286353]]
0.003745112336218197
-1745.390096920622

[[ 3.18218100e-03  2.13975900e+00  2.14699062e+00]
 [ 2.14739439e+00  1.22291710e-02  2.15423409e+00]
 [ 2.14251605e+00  2.12552583e+00 -1.28493800e-03]]
0.0043711302237953035
-1748.0693179390735

[[ 0.00286959  2.24846656  2.19254178]
 [ 2.21329814  0.01186342  2.20056113]
 [ 2.20935308  2.23513279 -0.00345522]]
0.016501920934352866
-1749.5451023085493

[[ 0.00297799  2.27105573  2.35407908]
 [ 2.33000374  0.01248737  2.36199553]
 [ 2.32589314  2.25718043 -0.00300793]]
0.027688060943406506
-1750.599248325882

[[ 3.83210800e-03  2.42833932e+00  2.36618022e+00]
 [ 2.38385074e+00  1.41629990e-02  2.37425268e+00]
 [ 2.37802033e+00  2.41172344e+00 -8.29293000

In [34]:
i.copy()

Atoms(symbols='NaCl', pbc=True, cell=[[0.00396863, 2.390737891, 2.376953172], [2.384037483, 0.014440717, 2.384690014], [2.3776946, 2.37352375, 0.000189276]])

In [31]:
i.get_potential_energy()

-1752.1577976160108